# Scrape Rap Song Lyrics From Genius.com

In [7]:
GENIUS_API_TOKEN = 'aW0h5orSEm64_BQgGvxl7Zfyv9iJqIewJXrQYNjY9NohhA-FneZG3tImuGJXsWkP'

In [8]:
import requests
from bs4 import BeautifulSoup
import os
import re

### Get a list of song URL's for an artist

potential improvement: what if the artist name doesn't match the exact writing or even missplled?

In [9]:
## Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

## Get Genius.com song URL's from artist object
def request_song_url(artist_name, page_cap):
    page = 1
    songs = []  # to store song URL's

    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()

        ## Collect up to page_cap song objects from the artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
        
        ## Collect song URL's from song objects
        for song in song_info:
            if len(songs) < page_cap:
                url = song['result']['url']
                songs.append(url)
        
        if len(songs) == page_cap:
            break
        else:
            page += 1
        
    print("Found {} songs by {}".format(len(songs), artist_name))
    return songs


In [28]:
## Demo of request_song_url()
request_song_url('Roddy Ricch', 20)

Found 20 songs by Roddy Ricch


['https://genius.com/Roddy-ricch-the-box-lyrics',
 'https://genius.com/Mustard-and-roddy-ricch-ballin-lyrics',
 'https://genius.com/Roddy-ricch-high-fashion-lyrics',
 'https://genius.com/Roddy-ricch-every-season-lyrics',
 'https://genius.com/Roddy-ricch-die-young-lyrics',
 'https://genius.com/Roddy-ricch-start-wit-me-lyrics',
 'https://genius.com/Roddy-ricch-ricch-forever-lyrics',
 'https://genius.com/Roddy-ricch-down-below-lyrics',
 'https://genius.com/Roddy-ricch-tip-toe-lyrics',
 'https://genius.com/Roddy-ricch-war-baby-lyrics',
 'https://genius.com/Roddy-ricch-bacc-seat-lyrics',
 'https://genius.com/Marshmello-and-roddy-ricch-project-dreams-lyrics',
 'https://genius.com/Roddy-ricch-perfect-time-lyrics',
 'https://genius.com/Roddy-ricch-late-at-night-lyrics',
 'https://genius.com/Roddy-ricch-intro-lyrics',
 'https://genius.com/Roddy-ricch-boom-boom-room-lyrics',
 'https://genius.com/Roddy-ricch-fucc-it-up-lyrics',
 'https://genius.com/Roddy-ricch-big-stepper-lyrics',
 'https://geniu

### Fetch lyrics from the URL's

In [35]:
## Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='Lyrics__Container-sc-1ynbvzw-5').get_text('\n') # '\n' is used to change the next line of lyrics to a new line

    ## Remove identifiers like chorus, verse, etc.
    lyrics = re.sub(r'\[.*?\]', '', lyrics)

    ## Remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])

    return lyrics

In [37]:
## Demo of scrape_song_lyrics()
print(scrape_song_lyrics('https://genius.com/Roddy-ricch-intro-lyrics'))

Whoa, whoa, whoa
Whoa, whoa
Whoa, whoa, whoa
Whoa, whoa-whoa-whoa
From out the streets, became a millionaire
I know niggas started in the trap, and they still there
'Member I was robbin', I bought jewelry and I still wear it
Came a long way with all my niggas, and they still here
My big bro behind bars, fighting two hundred years
I got that call, I lost my dawg, and I don't know how to feel
Gotta stay on savage time, you know it's war time
I remember I ain't used to leave the house without my .45
I ain't got no time
'Member Pappy had the Mackie at the stop sign
Shit was too, too real
I got my Glock, and laid 'em down, nigga, you knew the deal
That's why I moved to the Hills, and I'm new to it
'Cause I ain't sure why my dawg had to make the news reel
Got promethazine in my eyes, I'm cryin' purple tears
The streets left me cold-hearted, they hurt me still
Uh, and that's why I'm on tour, that's why my house a resort
That's why I got the four by four
I remember when I was low, tryna trap i

### Loop through all URL's and write lyrics to one file

In [48]:
def write_lyrics_to_file(artist_name, song_count):
    f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode('utf8'))
    f.close()

    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))

In [51]:
## Demo of write_lyrics_to_file()
write_lyrics_to_file('Roddy Ricch', 10)


Found 10 songs by Roddy Ricch
Wrote 280 lines to file from 10 songs
